In [1]:
%pylab inline
import json
import pandas as pd
import numpy as np

Populating the interactive namespace from numpy and matplotlib


In [2]:
import re
lst = [u'विश्व', u'समाचार', u'अर्थ', u'स्वास्थ्य', u'विज्ञान', u'मनोरञ्जन', u'उपत्यका', u'खेलकुद', u' / वाणिज्य']
def correction(txt):
    return txt.replace(u'\u202f', '').replace(u'\u200d', '').strip()

def correct_ekantipur_title(txt):
    for w in lst:
        txt = re.sub(r'^%s'%w, '', txt)
    return txt

In [3]:
data = pd.read_json("../combined.json", lines=True)
data.title = data.title.apply(correction)
data.subtitle = data.subtitle.apply(correction)
data.description = data.description.apply(correction)
data.title.loc[data.source=='ekantipur'] = data[data.source=='ekantipur'].title.apply(correct_ekantipur_title).values
data.drop(['subtitle', 'url'], axis=1, inplace=True)
data.head()

,id,category,title,date_nepali,date_english,description,source
0,ekantipur_61424493,world,"अस्ट्रेलियाले थप १३ देशबाट कामदार भित्र्याउने,...","श्रावण १८, २०७६",2019/8/04,(अस्ट्रेलिया) — अस्ट्रेलियाले हाल प्रदान गर्दै...,ekantipur
1,ekantipur_84606642,world,सुरक्षा माग्दै हजारौं स्वास्थ्यकर्मी आन्दोलित,"श्रावण १६, २०७६",2019/8/02,अस्ट्रेलिया — कार्यस्थलमा सुरक्षाको प्रत्याभूत...,ekantipur
2,ekantipur_65651897,world,बस बममा ठोकिदा ३४ को मृत्यु,"श्रावण १५, २०७६",2019/8/01,"हेरात, अफगानिस्तान — अफगानिस्तानको फरास प्रान्...",ekantipur
3,ekantipur_63346336,world,मोदीको कडा कदम : जम्मु–कश्मीरका विशेषाधिकार खोसिए,"श्रावण २०, २०७६",2019/8/06,नयाँदिल्ली — भारत सरकारले उत्तरी राज्य जम्मु–क...,ekantipur
4,ekantipur_3510010,world,"अमेरिकाको ओहायोमा पनि गोलीबारी, ९ जनाको मृत्यु...","श्रावण १९, २०७६",2019/8/05,काठमाडौँ — अमेरिकाको ओहायो राज्यको डेटन भन्ने ...,ekantipur


In [4]:
data.groupby('source').size().reset_index(name='counts')

,source,counts
0,ekantipur,4685
1,onlinekhabar,3820
2,setopati,2066


## Building a classifier for ekantipur for 5 categories

In [5]:
df = data.query("source == 'ekantipur' ")
df.groupby('category').size().reset_index(name='counts')

,category,counts
0,business,790
1,entertainment,250
2,health,39
3,national,243
4,news,2384
5,sports,606
6,technology,21
7,world,352


In [6]:
cats = ["business", "entertainment", "national", "sports", "world"]
df = df.query("category == @cats")
df.head()

,id,category,title,date_nepali,date_english,description,source
0,ekantipur_61424493,world,"अस्ट्रेलियाले थप १३ देशबाट कामदार भित्र्याउने,...","श्रावण १८, २०७६",2019/8/04,(अस्ट्रेलिया) — अस्ट्रेलियाले हाल प्रदान गर्दै...,ekantipur
1,ekantipur_84606642,world,सुरक्षा माग्दै हजारौं स्वास्थ्यकर्मी आन्दोलित,"श्रावण १६, २०७६",2019/8/02,अस्ट्रेलिया — कार्यस्थलमा सुरक्षाको प्रत्याभूत...,ekantipur
2,ekantipur_65651897,world,बस बममा ठोकिदा ३४ को मृत्यु,"श्रावण १५, २०७६",2019/8/01,"हेरात, अफगानिस्तान — अफगानिस्तानको फरास प्रान्...",ekantipur
3,ekantipur_63346336,world,मोदीको कडा कदम : जम्मु–कश्मीरका विशेषाधिकार खोसिए,"श्रावण २०, २०७६",2019/8/06,नयाँदिल्ली — भारत सरकारले उत्तरी राज्य जम्मु–क...,ekantipur
4,ekantipur_3510010,world,"अमेरिकाको ओहायोमा पनि गोलीबारी, ९ जनाको मृत्यु...","श्रावण १९, २०७६",2019/8/05,काठमाडौँ — अमेरिकाको ओहायो राज्यको डेटन भन्ने ...,ekantipur


80-20 train test split

In [20]:
from sklearn.model_selection import train_test_split
idxs_tr, idxs_te = train_test_split(np.arange(len(df)), test_size = 0.2, random_state = 1, stratify = df.source.values)

In [21]:
df.iloc[idxs_tr].groupby('category').size().reset_index(name='counts')

,category,counts
0,business,637
1,entertainment,193
2,national,189
3,sports,489
4,world,284


In [22]:
df.iloc[idxs_te].groupby('category').size().reset_index(name='counts')

,category,counts
0,business,153
1,entertainment,57
2,national,54
3,sports,117
4,world,68


## Fasttext vectors

In [97]:
# Entire news text is too long, so we will just take first 3 sentences (lead of an article)
def lead(txt):
#     sents = txt.split(u'।')
#     return u' । '.join(sents[:5])
    return txt

In [98]:
T_tr = df.iloc[idxs_tr].apply(lambda row: row["title"] + u' । ' + lead(row["description"]), axis=1).values
T_te = df.iloc[idxs_te].apply(lambda row: row["title"] + u' । ' + lead(row["description"]), axis=1).values
y_tr = df.iloc[idxs_tr].category.values
y_te = df.iloc[idxs_te].category.values

In [99]:
from fasttext.FastText import load_model, tokenize
model = load_model('/opt/installed/var/models/cc.ne.300.bin')

In [100]:
tokens_tr = [tokenize(txt) for txt in T_tr]
tokens_te = [tokenize(txt) for txt in T_te]

In [101]:
df.iloc[idxs_te[0]].title, lead(df.iloc[idxs_te[0]].description), tokens_te[0]

('कर्णालीवासीलाई दसैंमा जहाजको चिन्ता',
 'बाँके — दसैंमा घर जाने आशमा रहेका डोल्पाका प्रेम बुढा दिनहुँ यहाँस्थित विभिन्न विमानस्थलका कार्यालय धाउँछन् । टिकट नपाएपछि यसपालिको दसैं परिवारसँग मनाउन पाउने हो/होइन भन्ने कुराले उनलाई पिरोलेको छ ।गत आइतबार आफूसहित चारजनाको टिकट बोकेर यहाँस्थित वायु सेवा निगमको कार्यालयमा पुगेका थिए। १० दिनअघि काटेको टिकट देखाउँदै उनले भने, ‘निगमको जहाज औंशीपूर्णे पारेर आउँछ। त्यो पनि उड्ने हो कि होइन, भगवान् भरोसा बस्नुपर्छ।’ डोल्पा र सिमिकोटसम्म सडक नपुगेकाले दुर्गमका बासिन्दा महँगो टिकट काटेर जहाज चढ्न बाध्य छन्। कहिले खराब मौसमका कारण रद्द हुनु, समयमै नउड्नु, टिकट नपाउनु र व्यवस्थित उडान नहुँदा यात्रुले कष्ट भोग्नु परेको छ। दुर्गम क्षेत्रका जनताका लागि आशाको केन्द्र रहेको नेपाल वायु सेवा निगमले समेत आफ्नो नियमित उडान गर्न नसक्दा दसैंको मुखमा यात्रुले दुःख पाएका हुन्। निगमका स्टेसन म्यानेजर शंकरबहादुर केसीले विद्यार्थी र जागिरेहरू गन्तव्यमा पुग्न नसक्दा झगडा नै गर्ने गरेको सुनाए। उनीहरूलाई फ्लाइट रद्द हुनाको कारण सम्झाउन हम्मेहम्मे परेको पनि उनले बताए।  निग

In [102]:
lead(df.iloc[idxs_te[0]].description)

'बाँके — दसैंमा घर जाने आशमा रहेका डोल्पाका प्रेम बुढा दिनहुँ यहाँस्थित विभिन्न विमानस्थलका कार्यालय धाउँछन् । टिकट नपाएपछि यसपालिको दसैं परिवारसँग मनाउन पाउने हो/होइन भन्ने कुराले उनलाई पिरोलेको छ ।गत आइतबार आफूसहित चारजनाको टिकट बोकेर यहाँस्थित वायु सेवा निगमको कार्यालयमा पुगेका थिए। १० दिनअघि काटेको टिकट देखाउँदै उनले भने, ‘निगमको जहाज औंशीपूर्णे पारेर आउँछ। त्यो पनि उड्ने हो कि होइन, भगवान् भरोसा बस्नुपर्छ।’ डोल्पा र सिमिकोटसम्म सडक नपुगेकाले दुर्गमका बासिन्दा महँगो टिकट काटेर जहाज चढ्न बाध्य छन्। कहिले खराब मौसमका कारण रद्द हुनु, समयमै नउड्नु, टिकट नपाउनु र व्यवस्थित उडान नहुँदा यात्रुले कष्ट भोग्नु परेको छ। दुर्गम क्षेत्रका जनताका लागि आशाको केन्द्र रहेको नेपाल वायु सेवा निगमले समेत आफ्नो नियमित उडान गर्न नसक्दा दसैंको मुखमा यात्रुले दुःख पाएका हुन्। निगमका स्टेसन म्यानेजर शंकरबहादुर केसीले विद्यार्थी र जागिरेहरू गन्तव्यमा पुग्न नसक्दा झगडा नै गर्ने गरेको सुनाए। उनीहरूलाई फ्लाइट रद्द हुनाको कारण सम्झाउन हम्मेहम्मे परेको पनि उनले बताए।  निगमको जहाजले सातामा दुईपटक उडान भर्ने भए पन

In [103]:
X_tr = np.array([ np.mean(np.array([ model.get_word_vector(w) for w in article ] ), axis = 0 ) for article in T_tr] )
X_te = np.array([ np.mean(np.array([ model.get_word_vector(w) for w in article ] ), axis = 0 ) for article in T_te] )
X_tr.shape, X_te.shape

((1792, 300), (449, 300))

In [104]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import balanced_accuracy_score, make_scorer
bacc = make_scorer(balanced_accuracy_score)

In [105]:
params = {"gamma": [0.5, 1.0, 1.5,  2.0, 4.0], "C": [0.1, 1, 10, 100, 1000, 10000]}
svm = GridSearchCV(SVC(kernel = "rbf"), params, n_jobs = 6, cv = 3)
svm.fit(X_tr, y_tr)
print(svm.best_estimator_)
print(svm.score(X_te, y_te))

SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=4.0, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
0.7728285077951003


In [106]:
params = {"C": [0.1, 1, 10, 100, 1000, 10000]}
svml = GridSearchCV(SVC(kernel = "linear"), params, n_jobs = 6, cv = 3)
svml.fit(X_tr, y_tr)
print(svml.best_estimator_)
print(svml.score(X_te, y_te))

SVC(C=1000, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)
0.7683741648106904


In [107]:
from sklearn.linear_model import LogisticRegression
params = {"C": [0.1, 1, 10, 100, 1000, 10000]}
lr = LogisticRegression(penalty = "l2", solver = "liblinear", multi_class = "auto")
logreg = GridSearchCV(lr, params, n_jobs = 6, cv = 3)
logreg.fit(X_tr, y_tr)
print(logreg.best_estimator_)
print(logreg.score(X_te, y_te))

LogisticRegression(C=10000, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)
0.7817371937639198


## Clustering of ekantipur data subset (5 categories)

In [108]:
X_all = np.concatenate((X_tr, X_te) )
y_all = np.concatenate((y_tr, y_te) )
X_all.shape

(2241, 300)

In [109]:
from sklearn.cluster import KMeans, SpectralClustering

In [110]:
kmeans = KMeans(n_clusters=5, random_state=0).fit(X_all)
spectral = SpectralClustering(n_clusters=10, 
                                assign_labels="discretize",
                                random_state=0).fit(X_all)

In [111]:
from sklearn.metrics.cluster import *
homogeneity_score(y_all, kmeans.labels_), homogeneity_score(y_all, spectral.labels_)

(0.16281788289370858, 0.1807014787367899)

In [112]:
mutual_info_score(y_all, kmeans.labels_), mutual_info_score(y_all, spectral.labels_)

(0.2438228373769843, 0.2706038580085443)

In [113]:
completeness_score(y_all, kmeans.labels_), completeness_score(y_all, spectral.labels_)

(0.18571433381826377, 0.12284603560044073)

## Clustering entire data

In [ ]:
tokens = data.apply(lambda row: row["title"] + u' । ' + lead(row["description"]), axis=1).values
X = np.array([ np.mean(np.array([ model.get_word_vector(w) for w in article ] ), axis = 0 ) for article in tokens] )

In [ ]:
spectral = SpectralClustering(n_clusters=100, 
                                assign_labels="discretize",
                                random_state=0).fit(X)